In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")
import argparse
import concurrent.futures
import contextlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shutil
import spur
import subprocess
import sys
import json
import tensorflow as tf
import time
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
sns.set()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def create_data(clients, numofhours):
    for client in clients:
        command = f'python {path}client/clientServer-dataset.py -cl={client} -hr={numofhours}'
        command = command.split(' ')
        subprocess.check_output(command)

In [3]:
def run_clients(path, clients, numofhours, run):
    models = ['LR','RF', 'SVM']
    ##Run script
    results = {}
    for model in models:
        command = f'python {path}server/centralServer-{model}.py -cl={clients} -hr={numofhours} --run={run}'
        command = command.split(' ')
        output = subprocess.check_output(command) 
        results[model] = output.decode('utf-8')
    return results

In [4]:
def run_ensemble(path, clients, numofhours, run):
    ensemble_auc = []
    sample_size = []
    for client in clients:
        command = f'python {path}client/clientServer-Ensemble.py --client={client} -hr={numofhours} -rn={run}'
        command = command.split(' ')
        output = subprocess.check_output(command)
        output = [float(x) for x in output.decode('utf-8').split(',')]
        ensemble_auc.append(output[1])
        sample_size.append(output[0])
    return np.array(sample_size), np.array(ensemble_auc)

In [5]:
def averageResults(sample_size, ensemble_auc):
    relative_weights = sample_size / sum(sample_size)
    return np.sum(relative_weights * ensemble_auc)


In [9]:
clients = 'CUMC,UTH,Aachen'
path = '/Users/ae2722/Documents/DCI_code/'
clients_ = clients.split(',')
range_idx = np.flip(np.arange(0, 168, 12))
results, results_sites = {},[{},{},{}]

In [10]:
range_idx = np.flip(np.arange(0, 168, 12))
for i in range_idx[:-2]:
    results[i] = []
    for client, res in zip(clients_, results_sites):
        res[i] = []

In [ ]:
for i in range(300,400):
    for numofhours in range_idx[:-2]:
        model_results = run_clients(path,clients, numofhours, i)
        sample_size, ensemble_auc = run_ensemble(path, clients_, numofhours, i)
        auc = averageResults(sample_size, ensemble_auc)
        print(f'Ensemble AUC: {auc}')
        results[numofhours].append(auc)
        #client specific results
        for client, res in zip(clients_, results_sites):
            sample_size_, ensemble_auc_ = run_ensemble(path, [client,client], numofhours, i)
            auc_ = averageResults(sample_size_, ensemble_auc_)
            res[numofhours].append(auc_)
            print(f'{client} AUC: {auc_}')
    ##save each time
    pd.DataFrame.from_dict(results, orient = 'index').to_csv('results_.csv')
    for client, res in zip(clients_, results_sites):
        pd.DataFrame.from_dict(res, orient = 'index').to_csv(f'results_{client}.csv')
        

Ensemble AUC: 0.6875219040550848
CUMC AUC: 0.7397179788484136
UTH AUC: 0.4375
Aachen AUC: 0.375
Ensemble AUC: 0.641639351021502
CUMC AUC: 0.7344300822561691
UTH AUC: 0.28125
Aachen AUC: 0.4166666666666667
Ensemble AUC: 0.6532820005360052
CUMC AUC: 0.7115158636897767
UTH AUC: 0.59375
Aachen AUC: 0.4166666666666667
Ensemble AUC: 0.6652596532459232
CUMC AUC: 0.6774383078730905
UTH AUC: 0.875
Aachen AUC: 0.45833333333333337
Ensemble AUC: 0.6806079534912486
CUMC AUC: 0.7044653349001175
UTH AUC: 0.75
Aachen AUC: 0.45833333333333337
Ensemble AUC: 0.6697306574308863
CUMC AUC: 0.6668625146886016
UTH AUC: 0.84375
Aachen AUC: 0.5416666666666667
Ensemble AUC: 0.6785025872554477
CUMC AUC: 0.6645123384253819
UTH AUC: 0.90625
Aachen AUC: 0.6666666666666667
Ensemble AUC: 0.6410543839033542
CUMC AUC: 0.6509988249118683
UTH AUC: 0.71875
Aachen AUC: 0.41666666666666663
Ensemble AUC: 0.6160889150019584
CUMC AUC: 0.6615746180963571
UTH AUC: 0.75
Aachen AUC: 0.12499999999999997
Ensemble AUC: 0.6533644628610

Ensemble AUC: 0.675634444513163
CUMC AUC: 0.6780258519388953
UTH AUC: 0.6875
Aachen AUC: 0.625
Ensemble AUC: 0.5966071907147421
CUMC AUC: 0.6122209165687427
UTH AUC: 0.8125
Aachen AUC: 0.25
Ensemble AUC: 0.6402916073968706
CUMC AUC: 0.7038777908343126
UTH AUC: 0.5
Aachen AUC: 0.25
Ensemble AUC: 0.6262910507761765
CUMC AUC: 0.7144535840188013
UTH AUC: 0.25
Aachen AUC: 0.5833333333333334
Ensemble AUC: 0.6431623270868122
CUMC AUC: 0.6627497062279671
UTH AUC: 0.46875
Aachen AUC: 0.625
Ensemble AUC: 0.7015224606757787
CUMC AUC: 0.7420681551116334
UTH AUC: 0.4375
Aachen AUC: 0.4166666666666667
Ensemble AUC: 0.6997495206877358
CUMC AUC: 0.7690951821386603
UTH AUC: 0.40625
Aachen AUC: 0.5
Ensemble AUC: 0.6507514379367925
CUMC AUC: 0.6827262044653348
UTH AUC: 0.4375
Aachen AUC: 0.5
Ensemble AUC: 0.7344197744655411
CUMC AUC: 0.7074030552291422
UTH AUC: 1.0
Aachen AUC: 0.4166666666666667
Ensemble AUC: 0.6898025027315645
CUMC AUC: 0.6862514688601645
UTH AUC: 0.78125
Aachen AUC: 0.4583333333333333


CUMC AUC: 0.6850763807285546
UTH AUC: 0.625
Aachen AUC: 0.33333333333333337
Ensemble AUC: 0.741926422990496
CUMC AUC: 0.7226792009400704
UTH AUC: 0.65625
Aachen AUC: 0.8333333333333334
Ensemble AUC: 0.6515966767683015
CUMC AUC: 0.6756756756756757
UTH AUC: 0.75
Aachen AUC: 0.33333333333333337
Ensemble AUC: 0.653537118354052
CUMC AUC: 0.6621621621621622
UTH AUC: 0.8125
Aachen AUC: 0.20833333333333331
Ensemble AUC: 0.6146561321046446
CUMC AUC: 0.6345475910693301
UTH AUC: 0.78125
Aachen AUC: 0.25
Ensemble AUC: 0.6692255756901065
CUMC AUC: 0.6609870740305523
UTH AUC: 0.75
Aachen AUC: 0.12499999999999997
Ensemble AUC: 0.6581730471890654
CUMC AUC: 0.6768507638072856
UTH AUC: 0.78125
Aachen AUC: 0.20833333333333334
Ensemble AUC: 0.6709933617828355
CUMC AUC: 0.7079905992949471
UTH AUC: 0.8125
Aachen AUC: 0.5
Ensemble AUC: 0.6144834766116231
CUMC AUC: 0.6780258519388953
UTH AUC: 0.46875
Aachen AUC: 0.3333333333333333
Ensemble AUC: 0.653537118354052
CUMC AUC: 0.6668625146886016
UTH AUC: 0.5
Aache

CUMC AUC: 0.6198589894242068
UTH AUC: 0.8125
Aachen AUC: 0.16666666666666663
Ensemble AUC: 0.6462804337518298
CUMC AUC: 0.6980023501762632
UTH AUC: 0.59375
Aachen AUC: 0.41666666666666663
Ensemble AUC: 0.6659322365844105
CUMC AUC: 0.7079905992949471
UTH AUC: 0.4375
Aachen AUC: 0.5833333333333334
Ensemble AUC: 0.6575829261756034
CUMC AUC: 0.6968272620446533
UTH AUC: 0.4375
Aachen AUC: 0.5833333333333333
Ensemble AUC: 0.673521347434391
CUMC AUC: 0.7344300822561691
UTH AUC: 0.40625
Aachen AUC: 0.4166666666666667
Ensemble AUC: 0.6440874512956893
CUMC AUC: 0.7021151586368978
UTH AUC: 0.3125
Aachen AUC: 0.5
Ensemble AUC: 0.643414867957202
CUMC AUC: 0.6674500587544065
UTH AUC: 0.5625
Aachen AUC: 0.5
Ensemble AUC: 0.747830210072773
CUMC AUC: 0.7226792009400704
UTH AUC: 1.0
Aachen AUC: 0.6666666666666667
Ensemble AUC: 0.5899432040736389
CUMC AUC: 0.6216216216216217
UTH AUC: 0.6875
Aachen AUC: 0.375
Ensemble AUC: 0.6921655637330695
CUMC AUC: 0.6891891891891893
UTH AUC: 0.75
Aachen AUC: 0.7916666

Ensemble AUC: 0.7199991753767497
CUMC AUC: 0.7038777908343126
UTH AUC: 1.0
Aachen AUC: 0.625
Ensemble AUC: 0.6792576329189601
CUMC AUC: 0.725616921269095
UTH AUC: 0.71875
Aachen AUC: 0.33333333333333337
Ensemble AUC: 0.6571628837075061
CUMC AUC: 0.6750881316098707
UTH AUC: 0.75
Aachen AUC: 0.4583333333333333
Ensemble AUC: 0.6114504092192881
CUMC AUC: 0.6427732079905992
UTH AUC: 0.6875
Aachen AUC: 0.08333333333333331
Ensemble AUC: 0.6747917826293113
CUMC AUC: 0.6662749706227966
UTH AUC: 0.8125
Aachen AUC: 0.7083333333333335
Ensemble AUC: 0.6339677778464964
CUMC AUC: 0.6680376028202114
UTH AUC: 0.71875
Aachen AUC: 0.33333333333333337
Ensemble AUC: 0.6241006452676933
CUMC AUC: 0.6762632197414806
UTH AUC: 0.5625
Aachen AUC: 0.20833333333333334
Ensemble AUC: 0.6502437792483559
CUMC AUC: 0.7115158636897767
UTH AUC: 0.46875
Aachen AUC: 0.5
Ensemble AUC: 0.634135279444204
CUMC AUC: 0.6962397179788484
UTH AUC: 0.46875
Aachen AUC: 0.5
Ensemble AUC: 0.6561449893829756
CUMC AUC: 0.7497062279670974

...

CUMC AUC: 0.63572267920094
UTH AUC: 0.625
Aachen AUC: 0.45833333333333337
Ensemble AUC: 0.5786458655451792
CUMC AUC: 0.581668625146886
UTH AUC: 0.71875
Aachen AUC: 0.45833333333333337
Ensemble AUC: 0.5891855814624692
CUMC AUC: 0.6186839012925969
UTH AUC: 0.8125
Aachen AUC: 0.25
Ensemble AUC: 0.6981518131403714
CUMC AUC: 0.7162162162162162
UTH AUC: 0.59375
Aachen AUC: 0.7083333333333335
Ensemble AUC: 0.6552224421217555
CUMC AUC: 0.68213866039953
UTH AUC: 0.6875
Aachen AUC: 0.41666666666666663
Ensemble AUC: 0.6210650009277012
CUMC AUC: 0.6662749706227966
UTH AUC: 0.5
Aachen AUC: 0.5
Ensemble AUC: 0.6310145958315294
CUMC AUC: 0.6938895417156287
UTH AUC: 0.59375
Aachen AUC: 0.4583333333333333
Ensemble AUC: 0.581341352794442
CUMC AUC: 0.6304347826086956
UTH AUC: 0.53125
Aachen AUC: 0.3333333333333333
Ensemble AUC: 0.663401673985198
CUMC AUC: 0.7156286721504113
UTH AUC: 0.625
Aachen AUC: 0.4166666666666667
Ensemble AUC: 0.6125456119735297
CUMC AUC: 0.6480611045828437
UTH AUC: 0.3125
Aachen A

Ensemble AUC: 0.6507514379367925
CUMC AUC: 0.6868390129259695
UTH AUC: 0.59375
Aachen AUC: 0.5416666666666667
Ensemble AUC: 0.6908126662131238
CUMC AUC: 0.7109283196239717
UTH AUC: 0.5625
Aachen AUC: 0.45833333333333337
Ensemble AUC: 0.6613761931267652
CUMC AUC: 0.7250293772032902
UTH AUC: 0.46875
Aachen AUC: 0.5
Ensemble AUC: 0.6646643783371472
CUMC AUC: 0.7473560517038778
UTH AUC: 0.4375
Aachen AUC: 0.41666666666666663
Ensemble AUC: 0.6664398952728472
CUMC AUC: 0.6956521739130435
UTH AUC: 1.0
Aachen AUC: 0.12499999999999997
Ensemble AUC: 0.6758844084358957
CUMC AUC: 0.7279670975323149
UTH AUC: 0.75
Aachen AUC: 0.2916666666666667
Ensemble AUC: 0.5767904632321108
CUMC AUC: 0.5775558166862514
UTH AUC: 0.6875
Aachen AUC: 0.5
Ensemble AUC: 0.6306795926361143
CUMC AUC: 0.662749706227967
UTH AUC: 0.71875
Aachen AUC: 0.3333333333333333
Ensemble AUC: 0.6776573484239388
CUMC AUC: 0.6933019976498238
UTH AUC: 0.8125
Aachen AUC: 0.375
Ensemble AUC: 0.6073169851773972
CUMC AUC: 0.6451233842538191


CUMC AUC: 0.68096357226792
UTH AUC: 0.75
Aachen AUC: 0.16666666666666666
Ensemble AUC: 0.6529495742882471
CUMC AUC: 0.6327849588719153
UTH AUC: 0.84375
Aachen AUC: 0.5833333333333334
Ensemble AUC: 0.6572479229801885
CUMC AUC: 0.6721504112808461
UTH AUC: 0.78125
Aachen AUC: 0.33333333333333337
Ensemble AUC: 0.5783031315067928
CUMC AUC: 0.637485311398355
UTH AUC: 0.65625
Aachen AUC: 0.20833333333333334
Ensemble AUC: 0.6585106273321376
CUMC AUC: 0.699177438307873
UTH AUC: 0.6875
Aachen AUC: 0.375
Ensemble AUC: 0.7222720432102583
CUMC AUC: 0.7356051703877791
UTH AUC: 0.5625
Aachen AUC: 0.5416666666666667
Ensemble AUC: 0.6270512503350032
CUMC AUC: 0.6868390129259694
UTH AUC: 0.46875
Aachen AUC: 0.5
Ensemble AUC: 0.6183617828354672
CUMC AUC: 0.7056404230317274
UTH AUC: 0.40625
Aachen AUC: 0.2916666666666667
Ensemble AUC: 0.6441699136207145
CUMC AUC: 0.717978848413631
UTH AUC: 0.375
Aachen AUC: 0.375
Ensemble AUC: 0.6550523635763911
CUMC AUC: 0.7038777908343126
UTH AUC: 0.53125
Aachen AUC: 0.

Ensemble AUC: 0.6247758055538375
CUMC AUC: 0.6551116333725029
UTH AUC: 0.5
Aachen AUC: 0.2916666666666667
Ensemble AUC: 0.6729286494732719
CUMC AUC: 0.7585193889541715
UTH AUC: 0.5
Aachen AUC: 0.375
Ensemble AUC: 0.6325324180015256
CUMC AUC: 0.700352526439483
UTH AUC: 0.4375
Aachen AUC: 0.5
Ensemble AUC: 0.6211500402003836
CUMC AUC: 0.6504112808460634
UTH AUC: 0.65625
Aachen AUC: 0.5
Ensemble AUC: 0.655900179355557
CUMC AUC: 0.6551116333725029
UTH AUC: 0.8125
Aachen AUC: 0.45833333333333337
Ensemble AUC: 0.6264611293215411
CUMC AUC: 0.6745005875440658
UTH AUC: 0.625
Aachen AUC: 0.20833333333333331
Ensemble AUC: 0.6304270517657244
CUMC AUC: 0.6492361927144535
UTH AUC: 0.8125
Aachen AUC: 0.3333333333333333
Ensemble AUC: 0.6111952914012411
CUMC AUC: 0.6709753231492361
UTH AUC: 0.71875
Aachen AUC: 0.08333333333333334
Ensemble AUC: 0.6273888304780753
CUMC AUC: 0.6750881316098707
UTH AUC: 0.625
Aachen AUC: 0.2916666666666667
Ensemble AUC: 0.5940766281155297
CUMC AUC: 0.6245593419506463
UTH A

CUMC AUC: 0.6962397179788484
UTH AUC: 0.84375
Aachen AUC: 0.29166666666666663
Ensemble AUC: 0.7106345475910693
CUMC AUC: 0.7238542890716804
UTH AUC: 0.65625
Aachen AUC: 0.6666666666666667
Ensemble AUC: 0.6362457995753191
CUMC AUC: 0.6762632197414806
UTH AUC: 0.75
Aachen AUC: 0.41666666666666663
Ensemble AUC: 0.6360757210299545
CUMC AUC: 0.6921269095182139
UTH AUC: 0.6875
Aachen AUC: 0.2916666666666667
Ensemble AUC: 0.6285690725049992
CUMC AUC: 0.6797884841363102
UTH AUC: 0.46875
Aachen AUC: 0.45833333333333337
Ensemble AUC: 0.6649220731028511
CUMC AUC: 0.6874265569917744
UTH AUC: 0.4375
Aachen AUC: 0.7083333333333335
Ensemble AUC: 0.6364133011730265
CUMC AUC: 0.663924794359577
UTH AUC: 0.59375
Aachen AUC: 0.4166666666666667
Ensemble AUC: 0.656232605603315
CUMC AUC: 0.7168037602820211
UTH AUC: 0.46875
Aachen AUC: 0.5833333333333333
Ensemble AUC: 0.5672557569010658
CUMC AUC: 0.6139835487661575
UTH AUC: 0.4375
Aachen AUC: 0.33333333333333337
Ensemble AUC: 0.6919130228626795
CUMC AUC: 0.68

In [37]:
results = pd.read_csv('results_.csv', index_col = 0).T.to_dict(orient = 'list')
results_sites = []
for client in 'CUM':
    results_sites.append(pd.read_csv(f'results_{client}.csv', index_col = 0).T.to_dict(orient = 'list'))

In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('-s0','--centralServer', default = '')
    parser.add_argument('-cl','--clients', default = '')
    parser.add_argument('-pt','--path', default = '/Users/ae2722/Documents/DCI_code/')
    
    args = parser.parse_args()
    
    centralServer = args.centralServer
    clients = args.clients
    path = args.path
    
    clients_ = clients.split(',')
    range_idx = np.flip(np.arange(0, 168, 12))
    results = {}
    
    for numofhours in range_idx:
        create_data(clients, numofhours)
        results = run_clients(path,clients, numofhours)
        sample_size, ensemble_auc = run_ensemble(path, clients_, numofhours)
        auc = averageResults(sample_size, ensemble_auc)
        print(f'Ensemble AUC: {auc}')
        results[numofhours] = auc

if __name__ == '__main__':
    main()
    